<a href="https://colab.research.google.com/github/Auxilus08/Compiler-Design/blob/main/CD_prac_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Name : Akshat Tiwari**

**Section : A**

**Roll Number: A1-11**

In [5]:
import re
from collections import defaultdict

class ThreeAddressCodeOptimizer:
    def __init__(self, code):
        self.code = code.strip().split('\n')
        self.original_code = self.code.copy()

    def parse_instruction(self, instruction):
        # Handle assignment with operation: t1 = t2 + t3
        match = re.match(r'(\w+)\s*=\s*(\w+)\s*([+\-*/])\s*(\w+)', instruction)
        if match:
            return {
                'type': 'binary_op',
                'dest': match.group(1),
                'op1': match.group(2),
                'operator': match.group(3),
                'op2': match.group(4)
            }

        # Handle simple assignment: t1 = t2 or t1 = 5
        match = re.match(r'(\w+)\s*=\s*(\w+)', instruction)
        if match:
            return {
                'type': 'assignment',
                'dest': match.group(1),
                'value': match.group(2)
            }

        return None

    def is_constant(self, value):
        try:
            int(value)
            return True
        except ValueError:
            return False

    def copy_propagation(self):
        print("\n--- Copy Propagation ---")
        copy_map = {}
        optimized = []

        for instruction in self.code:
            parsed = self.parse_instruction(instruction)
            if not parsed:
                optimized.append(instruction)
                continue

            # Update copy map for simple assignments
            if parsed['type'] == 'assignment' and not self.is_constant(parsed['value']):
                copy_map[parsed['dest']] = parsed['value']

            # Apply copy propagation
            if parsed['type'] == 'binary_op':
                op1 = copy_map.get(parsed['op1'], parsed['op1'])
                op2 = copy_map.get(parsed['op2'], parsed['op2'])
                new_instruction = f"{parsed['dest']} = {op1} {parsed['operator']} {op2}"
                optimized.append(new_instruction)
            elif parsed['type'] == 'assignment':
                value = copy_map.get(parsed['value'], parsed['value'])
                new_instruction = f"{parsed['dest']} = {value}"
                optimized.append(new_instruction)
            else:
                optimized.append(instruction)

        self.code = optimized
        self.print_code()

    def constant_propagation(self):
        """Apply constant propagation optimization"""
        print("\n--- Constant Propagation ---")
        const_map = {}
        optimized = []

        for instruction in self.code:
            parsed = self.parse_instruction(instruction)
            if not parsed:
                optimized.append(instruction)
                continue

            # Track constants
            if parsed['type'] == 'assignment' and self.is_constant(parsed['value']):
                const_map[parsed['dest']] = parsed['value']

            # Apply constant propagation
            if parsed['type'] == 'binary_op':
                op1 = const_map.get(parsed['op1'], parsed['op1'])
                op2 = const_map.get(parsed['op2'], parsed['op2'])
                new_instruction = f"{parsed['dest']} = {op1} {parsed['operator']} {op2}"
                optimized.append(new_instruction)
                # Remove from const_map if dest is reassigned
                if parsed['dest'] in const_map:
                    del const_map[parsed['dest']]
            elif parsed['type'] == 'assignment':
                value = const_map.get(parsed['value'], parsed['value'])
                new_instruction = f"{parsed['dest']} = {value}"
                optimized.append(new_instruction)
            else:
                optimized.append(instruction)

        self.code = optimized
        self.print_code()

    def constant_folding(self):
        """Apply constant folding optimization"""
        print("\n--- Constant Folding ---")
        optimized = []

        for instruction in self.code:
            parsed = self.parse_instruction(instruction)
            if not parsed:
                optimized.append(instruction)
                continue

            if parsed['type'] == 'binary_op':
                if self.is_constant(parsed['op1']) and self.is_constant(parsed['op2']):
                    # Evaluate the expression
                    op1 = int(parsed['op1'])
                    op2 = int(parsed['op2'])

                    if parsed['operator'] == '+':
                        result = op1 + op2
                    elif parsed['operator'] == '-':
                        result = op1 - op2
                    elif parsed['operator'] == '*':
                        result = op1 * op2
                    elif parsed['operator'] == '/':
                        result = op1 // op2  # Integer division

                    new_instruction = f"{parsed['dest']} = {result}"
                    optimized.append(new_instruction)
                else:
                    optimized.append(instruction)
            else:
                optimized.append(instruction)

        self.code = optimized
        self.print_code()

    def common_subexpression_elimination(self):
        """Apply common subexpression elimination"""
        print("\n--- Common Subexpression Elimination ---")
        expr_map = {}  # Maps expressions to variables
        optimized = []

        for instruction in self.code:
            parsed = self.parse_instruction(instruction)
            if not parsed:
                optimized.append(instruction)
                continue

            if parsed['type'] == 'binary_op':
                # Create a normalized expression key
                expr_key = f"{parsed['op1']} {parsed['operator']} {parsed['op2']}"

                if expr_key in expr_map:
                    # Replace with existing computation
                    new_instruction = f"{parsed['dest']} = {expr_map[expr_key]}"
                    optimized.append(new_instruction)
                else:
                    # New expression, add to map
                    expr_map[expr_key] = parsed['dest']
                    optimized.append(instruction)
            else:
                optimized.append(instruction)

        self.code = optimized
        self.print_code()

    def dead_code_elimination(self):
        """Apply dead code elimination"""
        print("\n--- Dead Code Elimination ---")
        # Find all variables that are used
        used_vars = set()
        defined_vars = defaultdict(list)  # Maps variable to line numbers where it's defined

        # First pass: collect information
        for i, instruction in enumerate(self.code):
            parsed = self.parse_instruction(instruction)
            if not parsed:
                continue

            # Track definitions
            if 'dest' in parsed:
                defined_vars[parsed['dest']].append(i)

            # Track uses
            if parsed['type'] == 'binary_op':
                used_vars.add(parsed['op1'])
                used_vars.add(parsed['op2'])
            elif parsed['type'] == 'assignment' and 'value' in parsed:
                if not self.is_constant(parsed['value']):
                    used_vars.add(parsed['value'])

        # Second pass: eliminate dead code
        optimized = []
        for i, instruction in enumerate(self.code):
            parsed = self.parse_instruction(instruction)
            if not parsed:
                optimized.append(instruction)
                continue

            # Keep instruction if:
            # 1. The destination is used somewhere
            # 2. It's the last definition of a used variable
            if 'dest' in parsed:
                if parsed['dest'] in used_vars:
                    # Check if this is the last definition
                    defs = defined_vars[parsed['dest']]
                    if i == max(defs):
                        optimized.append(instruction)
                    else:
                        # Check if there's a later definition
                        later_defs = [d for d in defs if d > i]
                        if not later_defs:
                            optimized.append(instruction)
            else:
                optimized.append(instruction)

        self.code = optimized
        self.print_code()

    def print_code(self):
        """Print the current state of the code"""
        for instruction in self.code:
            print(instruction)

    def optimize(self):
        """Apply all optimization techniques until no further optimization is possible"""
        print("=== Original Three Address Code ===")
        self.print_code()

        iteration = 1
        while True:
            print(f"\n\n=== Optimization Iteration {iteration} ===")

            # Save current state
            prev_code = self.code.copy()

            # Apply optimizations in sequence
            self.copy_propagation()
            self.constant_propagation()
            self.constant_folding()
            self.common_subexpression_elimination()
            self.dead_code_elimination()

            # Check if code changed
            if prev_code == self.code:
                print("\n\nNo further optimizations possible!")
                break

            iteration += 1

        print("\n\n=== Final Optimized Three Address Code ===")
        self.print_code()

        print(f"\n\nOptimization Summary:")
        print(f"Original instructions: {len(self.original_code)}")
        print(f"Optimized instructions: {len(self.code)}")
        print(f"Instructions eliminated: {len(self.original_code) - len(self.code)}")


if __name__ == "__main__":
    tac_code = """
a = 5
b = 3
g = 9
c = a + b
d = a + b
e = c * 2
f = d * 2
g = 10
h = g + 1
i = h
j = i
k = 20
temp = k + 1
k = 30
result = e + f
"""

    optimizer = ThreeAddressCodeOptimizer(tac_code)
    optimizer.optimize()

=== Original Three Address Code ===
a = 5
b = 3
g = 9
c = a + b
d = a + b
e = c * 2
f = d * 2
g = 10
h = g + 1
i = h
j = i
k = 20
temp = k + 1
k = 30
result = e + f


=== Optimization Iteration 1 ===

--- Copy Propagation ---
a = 5
b = 3
g = 9
c = a + b
d = a + b
e = c * 2
f = d * 2
g = 10
h = g + 1
i = h
j = h
k = 20
temp = k + 1
k = 30
result = e + f

--- Constant Propagation ---
a = 5
b = 3
g = 9
c = 5 + 3
d = 5 + 3
e = c * 2
f = d * 2
g = 10
h = 10 + 1
i = h
j = h
k = 20
temp = 20 + 1
k = 30
result = e + f

--- Constant Folding ---
a = 5
b = 3
g = 9
c = 8
d = 8
e = c * 2
f = d * 2
g = 10
h = 11
i = h
j = h
k = 20
temp = 21
k = 30
result = e + f

--- Common Subexpression Elimination ---
a = 5
b = 3
g = 9
c = 8
d = 8
e = c * 2
f = d * 2
g = 10
h = 11
i = h
j = h
k = 20
temp = 21
k = 30
result = e + f

--- Dead Code Elimination ---
c = 8
d = 8
e = c * 2
f = d * 2
h = 11


=== Optimization Iteration 2 ===

--- Copy Propagation ---
c = 8
d = 8
e = c * 2
f = d * 2
h = 11

--- Constant Pr